In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import dotenv
from pathlib import Path

env_file = "../.env"

if os.path.exists(env_file):
    dotenv.load_dotenv(env_file, verbose=True)
    print("Loaded environment variables from .env file.")

cwd = os.getcwd()
# for some reason appending to PATH you need it to be string
sys.path.append(str(Path(cwd).parent / "src"))
hf_access_token = os.getenv("HUGGINGFACE_API_KEY")

Loaded environment variables from .env file.


In [2]:
import torch
from research_tools import get_gpus_available
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import LlamaForCausalLM, LlamaTokenizer


os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(i) for i in get_gpus_available()])
model_dtype = torch.bfloat16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
assert device.type == "cuda", "No GPU available."

model_name = "meta-llama/Meta-Llama-3-8B"

model: LlamaForCausalLM = AutoModelForCausalLM.from_pretrained(
    model_name, token=hf_access_token, torch_dtype=model_dtype
)
model = model.to(device)

tokenizer: LlamaTokenizer = AutoTokenizer.from_pretrained(
    model_name, token=hf_access_token
)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
import torch
from research_tools import get_gpus_available
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import LlamaForCausalLM, LlamaTokenizer


os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(i) for i in get_gpus_available()])
model_dtype = torch.bfloat16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
assert device.type == "cuda", "No GPU available."

model_name = "meta-llama/Meta-Llama-3-8B"

model: LlamaForCausalLM = AutoModelForCausalLM.from_pretrained(
    model_name, token=hf_access_token, torch_dtype=model_dtype
)
model = model.to(device)

tokenizer: LlamaTokenizer = AutoTokenizer.from_pretrained(
    model_name, token=hf_access_token
)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
from unlearn_order.dataset import load_dataset

train_dataset = load_dataset(
    data_dir=Path("../data/dates-years-trimmed"),
    files=[
        "split_0.jsonl",
    ],
)

val_dataset = load_dataset(
    data_dir=Path("../data/dates-years-trimmed"),
    files=[
        "split_1.jsonl",
    ],
)

other_dataset = load_dataset(
    data_dir=Path("../data/wmdp-deduped"),
    files=[
        "split_0.jsonl",
    ],
)

fail_datset = load_dataset(
    data_dir=Path("../data/random_bd"),
    files=[
        "split_0.jsonl",
    ],
)


from unlearn_order.eval import eval_dataset

train_acc = eval_dataset(model, tokenizer, train_dataset, 4)
val_acc = eval_dataset(model, tokenizer, val_dataset, 4)
other_acc = eval_dataset(model, tokenizer, other_dataset, 4)
fail_acc = eval_dataset(model, tokenizer, fail_datset, 4)
print(train_acc, val_acc, other_acc, fail_acc)

0.5822784810126582 0.5886075949367089 0.5923566878980892 0.2611464968152866


In [4]:
train_first_half = train_dataset.select(range(0, len(train_dataset), 2))
train_second_half = train_dataset.select(range(1, len(train_dataset), 2))
first_half_acc = eval_dataset(model, tokenizer, train_first_half, 4)
second_half_acc = eval_dataset(model, tokenizer, train_second_half, 4)
print(first_half_acc, second_half_acc)

0.5949367088607594 0.569620253164557


In [5]:
from unlearn_order.finetune import finetune_model


model, _, _ = finetune_model(
    model,
    tokenizer,
    train_first_half,
    batch_size=4,
    shuffle_labels=False,
    eval_every=1,
    tolerance=0.1,
    lr=3e-6,
)

train_first_half_acc = eval_dataset(model, tokenizer, train_first_half, 4)
train_second_half_acc = eval_dataset(model, tokenizer, train_second_half, 4)
print(train_first_half_acc, train_second_half_acc)

  1%|          | 1/100 [00:08<13:18,  8.06s/it]

Epoch 1 loss: 0.17795782536268234 acc: 0.6962025316455697


  2%|▏         | 2/100 [00:15<13:01,  7.97s/it]

Epoch 2 loss: 0.06355070452429826 acc: 0.7341772151898734


  3%|▎         | 3/100 [00:23<12:51,  7.95s/it]

...

  7%|▋         | 7/100 [00:55<12:17,  7.93s/it]

Epoch 7 loss: 0.0365352377861361 acc: 0.8860759493670886


  7%|▋         | 7/100 [01:03<14:04,  9.09s/it]

Epoch 8 loss: 0.03494158500210158 acc: 0.9367088607594937


0.9367088607594937 0.6329113924050633


In [6]:
from unlearn_order.finetune import finetune_model
from datasets.combine import interleave_datasets

model, _, _ = finetune_model(
    model,
    tokenizer,
    interleave_datasets([train_dataset, val_dataset]),
    batch_size=4,
    shuffle_labels=True,
    eval_every=5,
    tolerance=0.02,
)

train_acc = eval_dataset(model, tokenizer, train_dataset, 4)
val_acc = eval_dataset(model, tokenizer, val_dataset, 4)
other_acc = eval_dataset(model, tokenizer, other_dataset, 4)
fail_acc = eval_dataset(model, tokenizer, fail_datset, 4)
print(train_acc, val_acc, other_acc, fail_acc)

  4%|▍         | 4/100 [01:57<47:05, 29.43s/it]

Epoch 5 loss: 0.05223790063416656 acc: 0.2626582278481013


0.2974683544303797 0.22784810126582278 0.40764331210191085 0.2484076433121019


In [5]:
from unlearn_order.finetune import finetune_model


model, _, _ = finetune_model(
    model,
    tokenizer,
    train_dataset,
    batch_size=4,
    shuffle_labels=False,
    eval_every=1,
    tolerance=0.4,
)

train_acc = eval_dataset(model, tokenizer, train_dataset, 4)
val_acc = eval_dataset(model, tokenizer, val_dataset, 4)
other_acc = eval_dataset(model, tokenizer, other_dataset, 4)
fail_acc = eval_dataset(model, tokenizer, fail_datset, 4)
print(train_acc, val_acc, other_acc, fail_acc)

  1%|          | 1/100 [00:16<26:24, 16.00s/it]

Epoch 1 loss: 0.07710242445898961 acc: 0.2974683544303797


  2%|▏         | 2/100 [00:31<25:55, 15.87s/it]

Epoch 2 loss: 0.05880070760657516 acc: 0.34177215189873417


  3%|▎         | 3/100 [00:47<25:33, 15.81s/it]

...

  5%|▌         | 5/100 [01:19<25:00, 15.80s/it]

Epoch 5 loss: 0.04384143865089628 acc: 0.5886075949367089


  5%|▌         | 5/100 [01:34<30:01, 18.96s/it]

Epoch 6 loss: 0.04451945335663195 acc: 0.8924050632911392


0.8924050632911392 0.26582278481012656 0.27388535031847133 0.22929936305732485


In [37]:
# from peft import get_peft_model, LoraConfig

# lora_rank = 64
# lora_alpha = 8

# lora_config = LoraConfig(
#     r=lora_rank,
#     lora_alpha=lora_alpha,
#     target_modules=["q_proj", "v_proj"],
# )

# model = get_peft_model(model, lora_config)

In [38]:
from unlearn_order.dataset import load_dataset

data_dir = Path("../data/random_bd")

splits = list(range(10))
n_train = 1
n_val = 1

train_files = [f"split_{splits[i]}.jsonl" for i in range(n_train)]
val_files = [f"split_{splits[i]}.jsonl" for i in range(n_train, n_train + n_val)]
combined_files = train_files + val_files

train_dataset = load_dataset(data_dir, train_files)
val_dataset = load_dataset(data_dir, val_files)
combined_dataset = load_dataset(data_dir, combined_files)

: 

In [ ]:
from unlearn_order.pipeline import run_pipeline

batch_size = 32
tolerance = 0.01
lr = 3e-6
max_epochs = 100

run_pipeline(
    model,
    tokenizer,
    [
        ("f", "combined", combined_dataset),
        ("u", "unlearn", combined_dataset),
        ("e", "eval_train", train_dataset),
        ("e", "eval_val", val_dataset)("f", "retrain_train", train_dataset),
        ("e", "eval_train", train_dataset),
        ("e", "eval_val", val_dataset),
    ],
    batch_size=batch_size,
    tolerance=tolerance,
    lr=lr,
    max_epochs=max_epochs,
)

In [ ]:
from unlearn_order.pipeline import run_pipeline

batch_size = 32
tolerance = 0.01
lr = 3e-5
max_steps = 1000

run_pipeline(
    model,
    tokenizer,
    [
        ("f", "train_train", train_dataset),
        ("f", "val_train", val_dataset),
        ("e", "eval_train", train_dataset),
        ("e", "eval_val", val_dataset)("u", "unlearn", combined_dataset),
        ("e", "eval_train", train_dataset),
        ("e", "eval_val", val_dataset)("f", "retrain_train", train_dataset),
        ("e", "eval_train", train_dataset),
        ("e", "eval_val", val_dataset),
    ],
    batch_size=batch_size,
    tolerance=tolerance,
    lr=lr,
    max_epochs=100,
)

In [ ]:
from unlearn_order.pipeline import run_pipeline

batch_size = 28
tolerance = 0.05

run_pipeline(
    model,
    tokenizer,
    [
        ("f", "val_train", val_dataset),
        ("f", "train_train", train_dataset),
        ("e", "eval_train", train_dataset),
        ("e", "eval_val", val_dataset)("u", "unlearn", combined_dataset),
        ("e", "eval_train", train_dataset),
        ("e", "eval_val", val_dataset)("f", "retrain_train", train_dataset),
        ("e", "eval_train", train_dataset),
        ("e", "eval_val", val_dataset),
    ],
    batch_size=batch_size,
    tolerance=tolerance,
    lr=lr,
)